# Randomized Traveling Salesperson Problem - EDUC 689.99

The 20 most populous metropolitan areas in Canada are:

<ul>
  <li> Toronto, ON </li>
  <li> Montreal, QC </li>
  <li> Vancouver, BC </li>
  <li> Calgary, AB </li>
  <li> Edmonton, AB </li>
  <li> Ottawa-Gatineau, ON </li>
  <li> Winnipeg, MB </li>
  <li> Quebec City, QC </li>
  <li> Hamilton, ON </li>
  <li> Kitchener-Waterloo-Cambridge, ON </li>
  <li> London, ON </li>
  <li> Victoria, BC </li>
  <li> Halifax, NS </li>
  <li> Oshawa, ON </li>
  <li> Windsor, ON </li>
  <li> Saskatoon, SK </li>
  <li> St. Catharine's-Niagara, ON </li>
  <li> Regina, SK </li>
  <li> St. John's, NL </li>
  <li> Kelowna, BC </li>
</ul>
This notebook demonstrates the Traveling Salesperson problem (TSP) between these cities. This is done in two ways. First, the standard TSP is done $N$ times. Then, a *randomized* TSP is undertaken $N$ times. In the randomized TSP, the next city visited is the closest city 80% of the time. The other 20% of the time, the second closest city is chosen as the next destination. Both paths are plotted.


In [1]:
%%capture
# This cell magic suppresses output for this cell.
!pip install plotly --user;

In [8]:
from IPython.display import HTML
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import random
import csv

py.init_notebook_mode(connected=True)

hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).hide();
      }
    });
    $('div.output_prompt').css('opacity', 0);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Some raw code for this Jupyter notebook is hidden by default for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


# 3/8/18 For some reason I forgot to put in Victoria. I'm not going to fix it right now.

In [3]:
hide_me

def read_lines(filename, skip = False):
    """ Function to read in CSV file as numerical data. """
    with open(filename) as data:
        reader = csv.reader(data)
        if skip:  # Skip the header if it's there.
            next(reader, None)
            for row in reader:
                yield [ float(i) for i in row ]
        else:
            for row in reader:
                yield [ float(i) for i in row ]

# Names of the cities we'll be visiting.
city_names = ['tor','mtl','van','cgy','edm','ott','wpg','qbc','ham','kwc','ldn','hfx','osh','win','sks','stc','reg','stj','kel']

city_data = [row for row in read_lines('data/city_data.csv', True)]

def Cities(city_names):
    """ Function to construct a city given the numerical data in city_data. """
    def City(city_names, row_index):
        N = len(city_names)

        city = {}
        for index in range(N):
            city[city_names[index]] = city_data[row_index][index]

        return city

    # This builds the dictionary of cities that we want.
    cities = {}
    for i in range(len(city_names)):
        c = city_names[i]
        cities[c] = City(city_names,i)

    return cities

def Coordinates(city_names):
    """ Function to create a dictionary of dictionaries. Each dictionary 
        contains coordinates for a given city. """

    T = [i for i in read_lines('data/coordinates.csv')]

    """ This line returns a dictionary with each city name as the key. Each 
        key has a dictionary as its value. This second dictionary contains
        the coordinates for the respective city. """
    F = {city_names[i] : {'lat': T[i][0], 'lon': T[i][1]} for i in range(len(city_names))}

    return F

class Trip:
    """ Class for performing a trip. """
    
    def __init__(self, city_names, randomized):
        self.cities = Cities(city_names)
        self.total_distance = 0
        self.coordinates = Coordinates(city_names)
        self.latitudes = None
        self.longitudes = None
        self.randomized = randomized
        
        # This saves the order in which the cities were visited.
        self.order = []
        
        # This saves the city names in the order in which they were visited.
        self.cities_visited = []
        self.cities_visited_fullnames = None
        
        # Lookup table for full city names. This is used to display the city name on hover during plotting.
        self.city_lookup = {'tor': 'Toronto, ON', 
                            'mtl': 'Montreal, QC',
                            'van': 'Vancouver, BC',
                            'cgy': 'Calgary, AB',
                            'edm': 'Edmonton, AB',
                            'ott': 'Ottawa-Gatineau, ON',
                            'wpg': 'Winnipeg, MB',
                            'qbc': 'Quebec City, QC',
                            'ham': 'Hamilton, ON',
                            'kwc': 'Kitchener-Waterloo-Cambridge, ON',
                            'ldn': 'London, ON',
                            'hfx': 'Halifax, NS',
                            'osh': 'Oshawa, ON',
                            'win': 'Windsor, ON',
                            'sks': 'Saskatoon, SK',
                            'stc': "St. Catharine's, ON",
                            'reg': 'Regina, SK',
                            'stj': "St. John's, NL",
                            'kel': 'Kelowna, BC'
                           }
        
    def setup(self):
        """ Function to select a random starting point. """
        self.starting_city = random.choice(list(self.cities.keys()))
        self.cities_visited.append(self.starting_city)
#         print("We're starting in " + self.starting_city)
        
    def shortest(self):
        """ Function to return the closest city. """
        return min(self.cities[self.starting_city], key = self.cities[self.starting_city].get)
    
    def second_shortest(self):
        """ Function to return the second closest city. """
        values = sorted(self.cities[self.starting_city], key = self.cities[self.starting_city].get)
        
        if len(values) > 1:
            return values[1]
        else:
            return min(self.cities[self.starting_city], key = self.cities[self.starting_city].get)
        
#     m1, m2 = float('inf'), float('inf')
#     for x in values:
#         if x <= m1:
#             m1, m2 = x, m1
#         elif x < m2:
#             m2 = x
#     return m2

    def update(self):
        """ Function to move between cities. """
        # We don't want the starting city to be a destination for the other cities anymore.
        for key in self.cities.keys():
            self.cities[key].pop(self.starting_city)
        
        # If we run the randomized TSP.
        if self.randomized:
            if self.cities[self.starting_city] is not None:
                # Finds the second closest city to the current city 20% of the time.
                r = random.randint(0,99)
                
                # 7 Aug - can put in a later patch to accept the probability as a class parameter.
                if r < 80:
                    closest = self.shortest()
                else:
                    closest = self.second_shortest()    
        else:
            # Finds the closest city to the current city.
            if self.cities[self.starting_city] is not None:
                closest = self.shortest()
        
        # Add to the total distance.
        self.total_distance += self.cities[self.starting_city][closest]

        # Add the coordinates of the visited city and update the starting city.
        self.order.append(self.coordinates[self.starting_city])
        self.starting_city = closest
        
        # Add the current city to the list of visited cities.
        self.cities_visited.append(self.starting_city)
    
    def trip(self):
        """ Function to carry out a complete trip using recursion. """
        if len(self.cities[self.starting_city].keys()) > 1:
            self.update()
            self.trip()
        else:
            # Add the coordinates of the final city and get the latitude and longitude for plotting.
            self.order.append(self.coordinates[self.starting_city])
            self.cities_visited.append(self.starting_city)
            self.latitudes = [value['lat'] for value in self.order]
            self.longitudes = [value['lon'] for value in self.order]
            
            self.cities_visited_fullnames = [self.city_lookup[name] for name in self.cities_visited]
#             print('The total distance for this trip was {}.'.format(self.total_distance))
            
    def plot_trip(self):
        """ Function to plot the trip interactively with plotly. """
        journey = [ dict(
            type = 'scattergeo',
            name = "Path taken",
            lat = self.latitudes,
            lon = self.longitudes,
            hoverinfo = 'text',
            text = self.cities_visited_fullnames,
            mode = 'lines',
            line = dict(
                width = 2,
                color = 'rgb(0,0,0)',
            ),
        ) ]

        layout1 = dict(
                title = 'Shortest Traveling Salesperson Trip',
                showlegend = False,         
                geo = dict(
                    resolution = 50,
                    showland = True,
                    showlakes = True,
                    showocean = True,
                    landcolor = 'rgb(0, 179, 0)',
                    lakecolor = 'rgb(214, 224, 245)',
                    oceancolor = 'rgb(214, 224, 245)',
                    projection = dict( type="mercator" ),
                    coastlinewidth = 1,
                    lataxis = dict(
                        range = [ 40, 65 ],
                        showgrid = True,
                        tickmode = "linear",
                        dtick = 10
                    ),
                    lonaxis = dict(
                        range = [-135, -45],
                        showgrid = True,
                        tickmode = "linear",
                        dtick = 10
                    ),
                )
            )

        fig = dict(data = journey, layout=layout1)

        py.iplot(fig, validate=False)

In [4]:
hide_me
def takeNtrips(N):
    """ Take N trips and find the shortest trip distance. """
    # Initialize N trips.
    trips = [Trip(city_names, False) for i in range(N)]
    
    for trip in trips:
        trip.setup()
        trip.trip()
    
    distances = [trip.total_distance for trip in trips]
    
    shortest_trip = min(distances)
    result = trips[trip.total_distance == shortest_trip]
    
    print('The shortest trip was {} km.'.format(shortest_trip))
    result.plot_trip()
    
def takeNtrips_randomized(N):
    """ Take N trips and find the shortest trip distance. """
    # Initialize N trips.
    trips = [Trip(city_names, True) for i in range(N)]
    
    for trip in trips:
        trip.setup()
        trip.trip()
    
    distances = [trip.total_distance for trip in trips]
    
    shortest_trip = min(distances)
    result = trips[trip.total_distance == shortest_trip]
    
    print('The shortest trip was {} km.'.format(shortest_trip))
    result.plot_trip()

Let's start by taking 20 trips and plotting the fastest one (the least distance traveled). You can change the number of trips by changing the number in the parentheses.

In [5]:
takeNtrips(20)

The shortest trip was 8023.0 km.


Now let's compare the randomized strategy. We'll take 20 randomized TSP trips and plot the best result. Do you notice any differences?

In [7]:
takeNtrips_randomized(20)

The shortest trip was 9236.0 km.
